# accim: a Python library to perform building energy simulations considering adaptive comfort and data analytics

This Jupyter Notebook contains the code for the case study (Section 3 Results) of the research paper *accim: a Python library to perform building energy simulations considering adaptive comfort and data analytics*. It has been developed to ease the reproducibility of the paper, and the replicability with other IDF and EPW files or other arguments.

## 3. Results

## 3.1 Data pre-processing

### 3.1.1.	EPW files preparation

### 3.1.2.	ACCIS implementation

## 3.2.	Running simulations

## 3.3.	Data analysis

### 3.3.1 Tables

### 3.3.2.	Data visualization